In [ ]:
import pandas as pd
import numpy as np
import dcor


excel_file_path = ''
df = pd.read_excel(excel_file_path)


for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')


for col in df.columns:
    if df[col].isnull().any() and pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(df[col].mean())


df = df.replace([np.inf, -np.inf], np.nan)
for col in df.columns:
    if df[col].isnull().any() and pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(df[col].mean())

observed_column_name = df.columns[0] 
if isinstance(observed_column_name, (int, float)): 
    observed_column_name = "Observed_Trait_LL09" 
observed = df.iloc[:, 0]

method_names = [
    "GRSVNN",
    "LassoNet",
    "Blasso",
    "BRR",
    "BayesianCpi",
    "GRSVNN_pedi",
    "LassoNet_pedi",
    "Blasso_pedi",
    "BRR_pedi",
    "BayesianCpi_pedi"
]


dcor_results_list = []

for i in range(1, len(method_names) + 1): 
    
    predicted_column = df.iloc[:, i]

    current_method_name = method_names[i-1]

    
    if not pd.api.types.is_numeric_dtype(observed) or not pd.api.types.is_numeric_dtype(predicted_column):
        print(f"Skipping dCor for Method '{current_method_name}' due to non-numeric data after cleaning.")
        continue

    
    dcor_value = dcor.distance_correlation(observed, predicted_column)

    
    dcor_results_list.append({
        'Observed Trait': observed_column_name,
        'Prediction Method': current_method_name,
        'dCor Value': dcor_value
    })


dcor_df = pd.DataFrame(dcor_results_list)


output_file_path = ''
dcor_df.to_csv(output_file_path, index=False)

print(f"dCor values for method performance calculated and saved to {output_file_path}")
print("\nCalculated dCor values for Method Performance:")
print(dcor_df)